In [14]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression , Lasso , Ridge
from sklearn.metrics import mean_squared_error , r2_score , mean_absolute_error 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split , GridSearchCV ,RandomizedSearchCV
from scipy.stats import skew , shapiro , kstest , normaltest

import matplotlib.pyplot as plt
import seaborn as sns
import config1

In [15]:
df = pd.read_csv(config1.csv_file_path)
df = df.head(1000)
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7437100570,20140821T000000,291000.0,4,2.50,1860,6325,2.0,0,0,...,7,1860,0,1991,0,98038,47.3492,-122.030,1860,6449
996,8856004730,20140917T000000,199950.0,2,2.75,1590,20917,1.5,0,0,...,5,1590,0,1920,0,98001,47.2786,-122.250,1310,6000
997,3856902996,20140804T000000,553500.0,2,1.00,850,2340,1.0,0,0,...,7,850,0,1922,0,98105,47.6707,-122.328,1300,3000
998,1442800370,20150415T000000,189950.0,2,1.00,1030,4188,1.0,0,0,...,8,1030,0,1981,0,98038,47.3738,-122.057,1450,3376


In [16]:
df['date'] = df['date'].apply(lambda x : x.split('T')[0])

In [17]:
df['zipcode'].dtype

dtype('int64')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   int64  
 1   date           1000 non-null   object 
 2   price          1000 non-null   float64
 3   bedrooms       1000 non-null   int64  
 4   bathrooms      1000 non-null   float64
 5   sqft_living    1000 non-null   int64  
 6   sqft_lot       1000 non-null   int64  
 7   floors         1000 non-null   float64
 8   waterfront     1000 non-null   int64  
 9   view           1000 non-null   int64  
 10  condition      1000 non-null   int64  
 11  grade          1000 non-null   int64  
 12  sqft_above     1000 non-null   int64  
 13  sqft_basement  1000 non-null   int64  
 14  yr_built       1000 non-null   int64  
 15  yr_renovated   1000 non-null   int64  
 16  zipcode        1000 non-null   int64  
 17  lat            1000 non-null   float64
 18  long     

In [19]:
arr = np.array(df['date'])
arr

array(['20141013', '20141209', '20150225', '20141209', '20150218',
       '20140512', '20140627', '20150115', '20150415', '20150312',
       '20150403', '20140527', '20140528', '20141007', '20150312',
       '20150124', '20140731', '20140529', '20141205', '20150424',
       '20140514', '20140826', '20140703', '20140516', '20141120',
       '20141103', '20140626', '20141201', '20140624', '20150302',
       '20141110', '20141201', '20140624', '20141110', '20141203',
       '20140613', '20140528', '20141230', '20150213', '20140620',
       '20140715', '20140811', '20140707', '20141028', '20140729',
       '20140718', '20150325', '20140716', '20150428', '20150311',
       '20140916', '20150217', '20141231', '20150205', '20150303',
       '20140512', '20140819', '20150407', '20140827', '20150223',
       '20141210', '20140828', '20141021', '20141207', '20140603',
       '20141120', '20140909', '20141009', '20140825', '20140612',
       '20140912', '20150105', '20140610', '20140710', '201503

In [20]:
import datetime
import time

for i in range(len(arr)):
    arr[i] = datetime.datetime.strptime(arr[i] , '%Y%m%d').date()

In [21]:
for i in range(len(arr)):
    arr[i] -= datetime.datetime.strptime('20221230' , '%Y%m%d').date()

In [22]:
df['date'] = arr

In [23]:
df['date']

0     -3000 days
1     -2943 days
2     -2865 days
3     -2943 days
4     -2872 days
         ...    
995   -3053 days
996   -3026 days
997   -3070 days
998   -2816 days
999   -3019 days
Name: date, Length: 1000, dtype: timedelta64[ns]

In [24]:
df['date'] = df['date'].astype(str)
df['date'] = -(df['date'].apply(lambda x :(x.split()[0])).astype(int))

In [25]:
df['date'] = df['date'].astype(int)

In [39]:
x = df.drop(['id' , 'price' ,'date'] , axis = 1)
y = df['price']
x

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,2.50,1860,6325,2.0,0,0,4,7,1860,0,1991,0,98038,47.3492,-122.030,1860,6449
996,2,2.75,1590,20917,1.5,0,0,3,5,1590,0,1920,0,98001,47.2786,-122.250,1310,6000
997,2,1.00,850,2340,1.0,0,0,3,7,850,0,1922,0,98105,47.6707,-122.328,1300,3000
998,2,1.00,1030,4188,1.0,0,0,3,8,1030,0,1981,0,98038,47.3738,-122.057,1450,3376


In [40]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       1000 non-null   int64  
 1   bathrooms      1000 non-null   float64
 2   sqft_living    1000 non-null   int64  
 3   sqft_lot       1000 non-null   int64  
 4   floors         1000 non-null   float64
 5   waterfront     1000 non-null   int64  
 6   view           1000 non-null   int64  
 7   condition      1000 non-null   int64  
 8   grade          1000 non-null   int64  
 9   sqft_above     1000 non-null   int64  
 10  sqft_basement  1000 non-null   int64  
 11  yr_built       1000 non-null   int64  
 12  yr_renovated   1000 non-null   int64  
 13  zipcode        1000 non-null   int64  
 14  lat            1000 non-null   float64
 15  long           1000 non-null   float64
 16  sqft_living15  1000 non-null   int64  
 17  sqft_lot15     1000 non-null   int64  
dtypes: float6

In [41]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2 , random_state=40 )

### Linear Regression on Training and Testing

In [42]:
lin_reg_model = LinearRegression()
lin_reg_model

LinearRegression()

In [43]:
lin_reg_model.fit(x_train , y_train)
y_train_pred = lin_reg_model.predict(x_train)
# print('y_train_pred :' , y_train_pred)
print('train r2_score :' , r2_score(y_train_pred , y_train))
print('train MSE :' , mean_squared_error(y_train_pred , y_train))
print('train RMSE : ' , np.sqrt(mean_squared_error(y_train_pred , y_train)))
train_residual  =  y_train_pred - y_train
s1 = pd.Series(train_residual)
print('skewness factor :' , s1.skew())

train r2_score : 0.6321651863039535
train MSE : 32545151550.286793
train RMSE :  180402.74817831017
skewness factor : -1.724246612967635


In [44]:
y_test_pred = lin_reg_model.predict(x_test)
# print('y_train_pred :' , y_train_pred)
print('train r2_score :' , r2_score(y_test_pred , y_test))
print('train MSE :' , mean_squared_error(y_test_pred , y_test))
print('train RMSE : ' , np.sqrt(mean_squared_error(y_test_pred , y_test)))
test_residual  =  y_test_pred - y_test
s1 = pd.Series(test_residual)
print('skewness factor :' , s1.skew())

train r2_score : 0.6947795590582823
train MSE : 26154873727.699318
train RMSE :  161724.68496707396
skewness factor : -0.014807591153661747


In [116]:
lasso_model = Lasso(alpha = 1)

param_grid = {'alpha' : np.arange(0.05 , 1 , 0.001)}

gscv_lasso_model = GridSearchCV(lasso_model , param_grid , cv = 10)
gscv_lasso_model.fit(x_train , y_train)

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.972e+12, tolerance: 7.635e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.074e+13, tolerance: 9.108e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.650e+12, tolerance: 8.356e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.062e+13, tolerance: 8.993e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.081e+13, tolerance: 8.954e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.046e+13, tolerance: 8.860e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.650e+12, tolerance: 8.356e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.058e+13, tolerance: 9.116e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.040e+13, tolerance: 8.561e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.074e+13, tolerance: 9.108e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.650e+12, tolerance: 8.356e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.058e+13, tolerance: 9.116e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.972e+12, tolerance: 7.635e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.074e+13, tolerance: 9.108e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.650e+12, tolerance: 8.356e+09
  model = cd_fast.enet_coo

...

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.652e+12, tolerance: 8.356e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.058e+13, tolerance: 9.116e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.040e+13, tolerance: 8.561e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.089e+13, tolerance: 9.007e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.026e+13, tolerance: 8.534e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.972e+12, tolerance: 7.635e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.040e+13, tolerance: 8.561e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.062e+13, tolerance: 8.993e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.082e+13, tolerance: 8.954e+09
  model = cd_fast.enet_coo

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.973e+12, tolerance: 7.635e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.075e+13, tolerance: 9.108e+09
  model = cd_fast.enet_coordinate_descent(
D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.652e+12, tolerance: 8.356e+09
  model = cd_fast.enet_coo

GridSearchCV(cv=10, estimator=Lasso(alpha=1),
             param_grid={'alpha': array([0.05 , 0.051, 0.052, 0.053, 0.054, 0.055, 0.056, 0.057, 0.058,
       0.059, 0.06 , 0.061, 0.062, 0.063, 0.064, 0.065, 0.066, 0.067,
       0.068, 0.069, 0.07 , 0.071, 0.072, 0.073, 0.074, 0.075, 0.076,
       0.077, 0.078, 0.079, 0.08 , 0.081, 0.082, 0.083, 0.084, 0.085,
       0.086, 0.087, 0.088, 0.089, 0.09 , 0.091, 0.092, 0.093, 0.094,
       0.095, 0.096, 0....
       0.941, 0.942, 0.943, 0.944, 0.945, 0.946, 0.947, 0.948, 0.949,
       0.95 , 0.951, 0.952, 0.953, 0.954, 0.955, 0.956, 0.957, 0.958,
       0.959, 0.96 , 0.961, 0.962, 0.963, 0.964, 0.965, 0.966, 0.967,
       0.968, 0.969, 0.97 , 0.971, 0.972, 0.973, 0.974, 0.975, 0.976,
       0.977, 0.978, 0.979, 0.98 , 0.981, 0.982, 0.983, 0.984, 0.985,
       0.986, 0.987, 0.988, 0.989, 0.99 , 0.991, 0.992, 0.993, 0.994,
       0.995, 0.996, 0.997, 0.998, 0.999])})

In [117]:
gscv_lasso_model.best_estimator_

Lasso(alpha=0.9990000000000009)

In [118]:
gscv_lasso_model.best_params_

{'alpha': 0.9990000000000009}

In [119]:
ridge_model = Ridge(alpha = 1)
hyp = {'alpha' : np.arange(0.05 , 1 , 0.001)}

rdcv_model = RandomizedSearchCV(ridge_model , hyp , cv = 5 , random_state=40 , n_iter=100)
rdcv_model.fit(x_train,y_train)
rdcv_model.best_estimator_

Ridge(alpha=0.9900000000000009)

In [45]:
import pickle as pkl

In [46]:
x.head(1).T

,0
bedrooms,3.0000
bathrooms,1.0000
sqft_living,1180.0000
sqft_lot,5650.0000
floors,1.0000
waterfront,0.0000
view,0.0000
condition,3.0000
grade,7.0000
sqft_above,1180.0000


In [47]:
with open(config1.model_path,'wb') as f:
    pkl.dump(lin_reg_model, f)

In [48]:
x_train.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [49]:
y_train

970    532500.0
137    452000.0
82     355000.0
188    615000.0
358    900000.0
         ...   
440    227950.0
165    370000.0
7      291850.0
219    467000.0
326    274975.0
Name: price, Length: 800, dtype: float64

In [50]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       1000 non-null   int64  
 1   bathrooms      1000 non-null   float64
 2   sqft_living    1000 non-null   int64  
 3   sqft_lot       1000 non-null   int64  
 4   floors         1000 non-null   float64
 5   waterfront     1000 non-null   int64  
 6   view           1000 non-null   int64  
 7   condition      1000 non-null   int64  
 8   grade          1000 non-null   int64  
 9   sqft_above     1000 non-null   int64  
 10  sqft_basement  1000 non-null   int64  
 11  yr_built       1000 non-null   int64  
 12  yr_renovated   1000 non-null   int64  
 13  zipcode        1000 non-null   int64  
 14  lat            1000 non-null   float64
 15  long           1000 non-null   float64
 16  sqft_living15  1000 non-null   int64  
 17  sqft_lot15     1000 non-null   int64  
dtypes: float6